# Data Cleaning on tweets about Messi and Ronaldo, by Ibrahim SEROUIS 💻

## What is data cleaning ? 🧼

"Data cleaning is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset." [read more](https://www.tableau.com/learn/articles/what-is-data-cleaning)

## What to expect 🤔

In this Notebook, we're going to perform some data cleaning operations on [these tweets](https://www.kaggle.com/datasets/ibrahimserouis99/twitter-sentiment-analysis-and-word-embeddings).

# Libraries

In [1]:
!pip install --user clean-text

     |████████████████████████████████| 53 kB 380 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import re
import pandas as pd
from cleantext import clean

# Files

In [3]:
dataset_messi = pd.read_csv("/kaggle/input/twitter-sentiment-analysis-and-word-embeddings/messi_tweets.csv", encoding="utf-8")
dataset_ronaldo = pd.read_csv("/kaggle/input/twitter-sentiment-analysis-and-word-embeddings/ronaldo_tweets.csv", encoding="utf-8")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Display some samples

## Messi

In [4]:
dataset_messi.head(5)

,tweet_id,author_id,content,lang,date,source,geo,retweet_count,like_count,quote_count
0,1523587143632703488,1292468372618326023,@szn_united @Afc_Nick_ @Thogden Oh yeah and me...,en,2022-05-09T08:54:16.000Z,Twitter for Android,NaN,0,0,0
1,1523587126595825664,3985914141,@ws_onlyw @AbjaFCB @ShanviMessi This level of ...,en,2022-05-09T08:54:12.000Z,Twitter for iPhone,NaN,0,0,0
2,1523587117556715520,1480936861790924801,@FutSheriff Fred and messi tradeable. Thoughts...,en,2022-05-09T08:54:10.000Z,Twitter for Android,NaN,0,1,0
3,1523587113005948929,1424726769236529155,@NoCapFC @Mysticalleo_ @BecksRonnie their is a...,en,2022-05-09T08:54:09.000Z,Twitter Web App,NaN,0,0,0
4,1523587088381136896,942073809464467456,@goatssifreak Ok sir I think you know Messi be...,en,2022-05-09T08:54:03.000Z,Twitter for Android,NaN,0,0,0


## Ronaldo

In [5]:
dataset_ronaldo.head(5)

,tweet_id,author_id,content,lang,date,source,geo,retweet_count,like_count,quote_count
0,1523587244572872704,1383801795823116291,@markgoldbridge Where is the band leader? #Ron...,en,2022-05-09T08:54:40.000Z,Twitter for Android,NaN,0,0,0
1,1523587203351293953,1011901831411322880,@FutSheriff All untradeable except Alex Telles...,en,2022-05-09T08:54:30.000Z,Twitter Web App,NaN,0,0,0
2,1523587136968003584,288628771,@tshikolomot Or CR7 https://t.co/46rurMMMlW,en,2022-05-09T08:54:14.000Z,Twitter for iPhone,NaN,0,0,0
3,1523587113005948929,1424726769236529155,@NoCapFC @Mysticalleo_ @BecksRonnie their is a...,en,2022-05-09T08:54:09.000Z,Twitter Web App,NaN,0,0,0
4,1523587106647560192,278038673,🗣 “[Erik] ten hag needs some constance.” @Fle...,en,2022-05-09T08:54:07.000Z,Wildmoka,NaN,5,19,0


# Data cleaning 🧼🧽

## Messi

### Check for null values

In [6]:
print(f"Null values per column : \n\n{dataset_messi.isna().sum()}")

Null values per column : 

tweet_id              0
author_id             0
content               0
lang                  0
date                  0
source                0
geo              249522
retweet_count         0
like_count            0
quote_count           0
dtype: int64


### Drop duplicates
Based on tweet ID

In [7]:
print(f"Row count before deletion: {len(dataset_messi.index)} ")
dataset_messi = dataset_messi.drop_duplicates(subset="tweet_id", keep="first")
print(f"Row count after deletion: {len(dataset_messi.index)}")

Row count before deletion: 251530 
Row count after deletion: 213969


#### Percentage of null geo values 

In [8]:
count = len(dataset_messi.index)
print(f"Number of values : {count}")

Number of values : 213969


In [9]:
count_null = dataset_messi.isna().sum()["geo"]
percentage = count_null*100/count
print(f"Percentage of null geo values {round(percentage,2)}%")

Percentage of null geo values 99.18%


### Assign the -1 id to null geo values

In [10]:
dataset_messi.geo = dataset_messi.geo.apply(lambda x: -1 if pd.isna(x) else x)

#### Display results

In [11]:
dataset_messi["geo"].head(5)

0    -1
1    -1
2    -1
3    -1
4    -1
Name: geo, dtype: object

In [12]:
print(f"Some locations IDs: {dataset_messi['geo'].unique()[0:5]}")

Some locations IDs: [-1 '9d7cae88ff6a29f5' '01238f7f200d1e45' '009b4bcc18019797'
 '01983bef79d0f0cf']


### Clean tweets : remove mentions, extra spaces and links

#### Utility function

In [13]:
def clean_tweet(text):
    """
    Removes punctuation, emojis, normalizes whitespaces...from a text
    """
    
    text = clean(text,
                 no_punct=True,
                 lower=True,
                 no_emoji=True,
                 normalize_whitespace=True
                )
    
    return text

#### Create the text cleaning regular expressions 

In [14]:
# Remove mentions
regex_mentions = r"@[A-Za-z0-9_]+"
# Remove links
regex_links = r"https?://[A-Za-z0-9./]+"
# Remove some special characters
regex_special = r"[^A-Za-z0-9]+"
# Remove numbers 
regex_numbers = r"[0-9]+"
# Remove ordinals 
regex_ordinals = r"[0-9]+(?:st| st|nd| nd|rd| rd|th| th)"

### Clean tweets : remove mentions, links, special characters, extra spaces...

In [15]:
# Remove mentions
dataset_messi.content = dataset_messi.content.apply(lambda x: re.sub(regex_mentions, " ", str(x).strip()))
# Remove links 
dataset_messi.content = dataset_messi.content.apply(lambda x: re.sub(regex_links, " ", str(x).strip()))
# Remove special characters
dataset_messi.content = dataset_messi.content.apply(lambda x: re.sub(regex_special, " ", str(x).strip()))
# Remove ordinals
dataset_messi.content = dataset_messi.content.apply(lambda x: re.sub(regex_ordinals, " ", str(x).strip()))
# Remove numbers 
dataset_messi.content = dataset_messi.content.apply(lambda x: re.sub(regex_numbers, " ", str(x).strip()))
# Clean tweets
dataset_messi.content = dataset_messi.content.apply(lambda x: clean_tweet(x)) 

#### Display results

In [16]:
dataset_messi.head(5)

,tweet_id,author_id,content,lang,date,source,geo,retweet_count,like_count,quote_count
0,1523587143632703488,1292468372618326023,oh yeah and messi contributed so heavily to hi...,en,2022-05-09T08:54:16.000Z,Twitter for Android,-1,0,0,0
1,1523587126595825664,3985914141,this level of stretch is insane he literally t...,en,2022-05-09T08:54:12.000Z,Twitter for iPhone,-1,0,0,0
2,1523587117556715520,1480936861790924801,fred and messi tradeable thoughts,en,2022-05-09T08:54:10.000Z,Twitter for Android,-1,0,1,0
3,1523587113005948929,1424726769236529155,their is a reason most tickets sold in worldcu...,en,2022-05-09T08:54:09.000Z,Twitter Web App,-1,0,0,0
4,1523587088381136896,942073809464467456,ok sir i think you know messi better than mess...,en,2022-05-09T08:54:03.000Z,Twitter for Android,-1,0,0,0


### Check for non-english content

In [17]:
dataset_messi = dataset_messi[dataset_messi["lang"]=="en"]

In [18]:
assert dataset_messi["lang"].unique()=="en", "Non-english content detected"

> Test passed

## Ronaldo

### Check for null values

In [19]:
dataset_ronaldo.isna().sum()

tweet_id              0
author_id             2
content               2
lang                  5
date                  8
source                5
geo              248295
retweet_count         5
like_count            8
quote_count           8
dtype: int64

### Drop null values
For columns relevant to our analysis

In [20]:
dataset_ronaldo.dropna(subset=["lang","date","source","retweet_count","like_count", "quote_count"], inplace=True)

In [21]:
print(f"Null values: \n{dataset_ronaldo.isna().sum()}")

Null values: 
tweet_id              0
author_id             0
content               0
lang                  0
date                  0
source                0
geo              248290
retweet_count         0
like_count            0
quote_count           0
dtype: int64


### Drop duplicates

In [22]:
print(f"Row count before deletion: {len(dataset_ronaldo.index)} ")
dataset_ronaldo = dataset_ronaldo.drop_duplicates(subset="tweet_id", keep="first")
print(f"Row count after deletion: {len(dataset_ronaldo.index)}")

Row count before deletion: 251370 
Row count after deletion: 250670


### Percentage of null geo values

In [23]:
count = len(dataset_ronaldo.index)
print(f"Number of values: {count}")

Number of values: 250670


In [24]:
count_null = dataset_ronaldo.isna().sum()["geo"]
percentage = count_null * 100/count
print(f"Percentage of null geo values: {round(percentage,2)}%")

Percentage of null geo values: 98.77%


### Assign the -1 id to null geo values

In [25]:
dataset_ronaldo.geo = dataset_ronaldo.geo.apply(lambda x: -1 if pd.isna(x) else x)

#### Display results

In [26]:
dataset_ronaldo["geo"].head(5)

0    -1
1    -1
2    -1
3    -1
4    -1
Name: geo, dtype: object

In [27]:
print(f"Some location IDs: {dataset_ronaldo['geo'].unique()[0:5]}")

Some location IDs: [-1 '9d7cae88ff6a29f5' '73a5d83329504007' '0118c71c0ed41109'
 '01aca86a155c9099']


### Clean tweets : remove mentions, links, special characters, extra spaces...

In [28]:
# Remove mentions
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: re.sub(regex_mentions, " ", str(x).strip()))
# Remove links
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: re.sub(regex_links, " ", str(x).strip()))
# Remove special characters
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: re.sub(regex_special, " ", str(x).strip()))
# Remove ordinals
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: re.sub(regex_ordinals, " ", str(x).strip()))
# Remove numbers
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: re.sub(regex_numbers, " ", str(x).strip()))
# Clean tweets
dataset_ronaldo.content = dataset_ronaldo.content.apply(lambda x: clean_tweet(x)) 

#### Display results

In [29]:
dataset_ronaldo.head(5)

,tweet_id,author_id,content,lang,date,source,geo,retweet_count,like_count,quote_count
0,1523587244572872704,1383801795823116291,where is the band leader ronaldo,en,2022-05-09T08:54:40.000Z,Twitter for Android,-1,0,0,0
1,1523587203351293953,1011901831411322880,all untradeable except alex telles what could ...,en,2022-05-09T08:54:30.000Z,Twitter Web App,-1,0,0,0
2,1523587136968003584,288628771,or cr,en,2022-05-09T08:54:14.000Z,Twitter for iPhone,-1,0,0,0
3,1523587113005948929,1424726769236529155,their is a reason most tickets sold in worldcu...,en,2022-05-09T08:54:09.000Z,Twitter Web App,-1,0,0,0
4,1523587106647560192,278038673,erik ten hag needs some constance says he want...,en,2022-05-09T08:54:07.000Z,Wildmoka,-1,5,19,0


### Check for non-english content

In [30]:
dataset_ronaldo = dataset_ronaldo[dataset_ronaldo["lang"]=="en"]

In [31]:
assert dataset_ronaldo["lang"].unique()=="en", "Non-english content detected"

```
Test passed
```

# Save the cleaned datasets 💾

> Note : Index = false tells the Pandas library not to add an index when writing the file. In our case, since there's already an index, no need to write a new one. 

## Messi 

In [32]:
dataset_messi.to_csv("Cleaned_messi_tweets.csv", index=False)

## Ronaldo

In [33]:
dataset_ronaldo.to_csv("Cleaned_ronaldo_tweets.csv", index=False)

# Thank you for your time 😄